# Bilan Hydrique Climatologie

In [1]:
import numpy as np
import pandas as pd

# Définition de la station de référence
REF_STATION_NAME = 'La Petite Claye'
REF_STATION_LATLON = [48.541356, -1.615400]
# Altitude (m)
REF_STATION_ALTITUDE = 50.

# Identification de l'API Météo-France
APPLICATION_ID = 'ZlFGb1VCNzdlQ3c5QmhSMU1IbE8xQTluOE0wYTpUS3l1YkcweGJmSTJrQlJVaGNiSkNHTXczdHNh'

# Météo-France API
METEOFRANCE_API = 'DPClim'

# Départements où chercher des stations
ID_DEPARTEMENTS = [35, 50]

# Fréquence des données climatiques
METEOFRANCE_FREQUENCE = 'quotidienne'

# Liste des variables météorologiques utilisées pour le bilan hydrique
LISTE_VARIABLES_METEO = ['ETPGRILLE', 'RR']

# NN_NOMBRE = 20
# NN_RAYON_KM = None
NN_NOMBRE = None
NN_RAYON_KM = 20.

In [2]:
import meteofrance

# Initialisation d'un client pour accéder à l'API Météo-France
client = meteofrance.Client(APPLICATION_ID, METEOFRANCE_API)

# Issue a series of API requests an example. For use this test, you must first subscribe to the arome api with your application
client.session.headers.update({'Accept': '*/*'})

In [3]:
LECTURE_LISTE_STATIONS = True

l_listes = []
for id_dep in ID_DEPARTEMENTS:
    filepath_liste_stations = meteofrance.get_filepath_liste_stations(
        client, id_dep)
    
    if LECTURE_LISTE_STATIONS:
        # Lecture par département
        df_liste_stations_dep = pd.read_csv(
            filepath_liste_stations, index_col=client.id_station_label)
    else:
        # Demande de la liste des stations pour le département
        section = meteofrance.SECTION_LISTE_STATIONS
        params = {'id-departement': id_dep}
        response = meteofrance.demande(
            client, section, params=params, frequence=METEOFRANCE_FREQUENCE)
        df_liste_stations_dep = meteofrance.response_text_to_frame(
            client, response, index_col=client.id_station_label)

        # Sauvegarde par département
        df_liste_stations_dep.to_csv(filepath_liste_stations)

    # Liste pour compilation
    l_listes.append(df_liste_stations_dep)

# Compilation
df_liste_stations_brute = pd.concat(l_listes, axis='index')

# Garder les stations valides seulement
df_liste_stations = meteofrance.filtrer_stations_valides(client, df_liste_stations_brute)

In [4]:
import geo

df_liste_stations_nn = geo.selection_plus_proches_voisins(
    df_liste_stations, REF_STATION_LATLON, client.latlon_labels,
    nombre=NN_NOMBRE, rayon_km=NN_RAYON_KM)

df_liste_stations_nn

,nom,typePoste,lon,lat,alt,distance
id,,,,,,
35044001,BROUALAN,2,-1.640833,48.485667,99,6
50410003,PONTORSON,1,-1.505167,48.585667,33,9
35224001,PLERGUER,2,-1.843667,48.524833,30,17


In [5]:
DATE_DEB_PERIODE = '2024-01-01T00:00:00Z'
DATE_FIN_PERIODE = '2024-12-31T23:00:00Z'

LECTURE_DONNEE = False

filepath_donnee = meteofrance.get_filepath_donnees_periode(
    client, df_liste_stations_nn, DATE_DEB_PERIODE, DATE_FIN_PERIODE)
    
if LECTURE_DONNEE:
    # Lecture des données des stations pour la période
    df_meteo = pd.read_csv(
        filepath_donnee,
        index_col=[client.id_station_donnee_label, client.time_label])
else:
    # Demande des données des stations pour la période
    df_meteo = meteofrance.compiler_telechargement_des_stations_periode(
        client, df_liste_stations_nn, DATE_DEB_PERIODE, DATE_FIN_PERIODE,
        frequence=METEOFRANCE_FREQUENCE)[LISTE_VARIABLES_METEO]
    
    # Sauvegarde par département
    df_meteo.to_csv(filepath_donnee)

35044001 
 201
50410003 
 201
Received status code 204. Retrying...
35224001 
 201


In [11]:
LECTURE_DONNEE_REF = False

str_ref_station_name = REF_STATION_NAME.lower().replace(' ', '')
filepath_donnee_ref = filepath_donnee.with_name(
    filepath_donnee.stem + '_' + str_ref_station_name + filepath_donnee.suffix)

if LECTURE_DONNEE_REF:
    # Lecture des données des stations pour la période
    df_meteo_ref = pd.read_csv(
        filepath_donnee_ref, index_col=client.time_label)
else:
    # Demande des données des stations pour la période
    df_meteo_ref = geo.interpolation_inverse_distance_carre(
        df_meteo, df_liste_stations_nn['distance'])
    
    # Sauvegarde par département
    df_meteo_ref.to_csv(filepath_donnee_ref)

In [12]:
# Renommer les variables
df_meteo_ref = df_meteo_ref.rename(columns=client.inv_variables_labels)

In [13]:
# RFU finale cible (mm)
RFU_CIBLE = None

# Fraction du sol occupé par des cailloux et graviers (entre 0 pour absence de cailloux et 1 pour totalité de cailloux)
FRACTION_CAILLOUX = 0.1

# Choix de la texture
TEXTURE = 'Terres limoneuses'

# Coefficient de conversion de la RU en RFU (entre 1/2 et 2/3)
RU_VERS_RFU = 2. / 3

# Fraction de la réserve utile du sol remplie d'eau (entre 0 pour une période sèche et 1 pour une période pluvieuse)
FRACTION_REMPLIE = 1.

In [14]:
# Choix de la culture
CULTURE = 'Pomme de terre'

# Choix du stade
STADE = 'Vegetation'

In [15]:
import bilan

df_bilan = bilan.calcul_bilan(
    TEXTURE, FRACTION_CAILLOUX,
    CULTURE, STADE,
    df_meteo_ref,
    rfu_cible=RFU_CIBLE, fraction_remplie=FRACTION_REMPLIE, ru_vers_rfu=RU_VERS_RFU)

df_bilan

,ru,profondeur_terrefine,profondeur_enracinement,rfu,etm_culture,rfu_cible,precipitation,besoin_irrigation,irrigation
DATE,,,,,,,,,
2024-01-01,48.6,27.0,30.0,32.4,0.925494,32.4,11.407057,-10.481563,False
2024-01-02,48.6,27.0,30.0,32.4,1.023411,32.4,10.106273,-9.082862,False
2024-01-03,48.6,27.0,30.0,32.4,1.523627,32.4,1.996128,-0.472502,False
2024-01-04,48.6,27.0,30.0,32.4,0.926266,32.4,1.300858,-0.374592,False
2024-01-05,48.6,27.0,30.0,32.4,0.669784,32.4,6.254496,-5.584712,False
...,...,...,...,...,...,...,...,...,...
2024-12-27,48.6,27.0,30.0,32.4,0.187145,32.4,0.127469,0.059677,True
2024-12-28,48.6,27.0,30.0,32.4,0.104291,32.4,0.127469,-0.023178,False
2024-12-29,48.6,27.0,30.0,32.4,0.000000,32.4,0.000000,0.000000,False
